# Deploy Endpoint

# dev account

Login AWS:
* account: mlops-dev
* user: dev-developers
* cloud9: my-mlops-works


# Prod SageMaker Endpoint Configuration

* mkdir prod-deploy-endpoint
* aws --profile mlops-devops-developers codecommit create-repository --repository-name prod-deploy-endpoint
* cd prod-deploy-endpoint
* git init
* git remote add origin https://git-codecommit.us-west-2.amazonaws.com/v1/repos/prod-deploy-endpoint
* git remote -v
* touch prod-deploy-endpoint.py

### prod-deploy-endpoint.py

In [ ]:
import boto3, sys

prod_access_key = sys.argv[1]
prod_secrete_key = sys.argv[2]
model_name = sys.argv[3]

# prod sagemaker
sess_prod = boto3.Session(aws_access_key_id=prod_access_key, aws_secret_access_key=prod_secrete_key)
sm_prod = sess_prod.client('sagemaker')
prod_account = sess_prod.client('sts').get_caller_identity()['Account']

resp = sm_prod.create_endpoint_config(EndpointConfigName=model_name,
    ProductionVariants=[
        {
            'VariantName': 'prod',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.t2.medium'
        }
    ])
print(resp['EndpointConfigArn'])

resp = sm_prod.create_endpoint(
    EndpointName=model_name,
    EndpointConfigName=model_name
)

print(resp['EndpointArn'])

# DevOps

* Please login as mlops-devops-admin user

## CodeBuild

* project name: prod-deploy-endpoint
* image: amazonlinux2
* role: codebuild-prod-deploy-endpoint-service-role

* Buildspec.yml:

In [ ]:
version: 0.2

env:
  variables:
     model_name: "<model_name>"
  secrets-manager:
     prod_access_key: prod/sagemaker:prod-sagemaker-access
     prod_secret_key: prod/sagemaker:prod-sagemaker-secret
phases:
  install:
    runtime-versions:
      python: 3.8
  build:
    commands:
      - python prod-deploy-endpoint.py $prod_access_key $prod_secret_key $model_name   

## IAM policy

* service role: codebuild-prod-deploy-endpoint-service-role
* inline policy: codebuild-prod-deploy-endpoint

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "secretsmanager:GetSecretValue",
            "Resource": "arn:aws:secretsmanager:*:*:secret:prod/sagemaker-*"
        }
    ]
}

# Build codebuild

* variable name
    * key: model_name

In [ ]:
[Container] 2020/01/07 15:34:50 Running command python prod-deploy-endpoint.py $prod_access_key $prod_secret_key $model_name 
arn:aws:sagemaker:us-west-2:***:endpoint-config/scikit-bring-your-own-2020-01-06-12-47-23 
arn:aws:sagemaker:us-west-2:***:endpoint/scikit-bring-your-own-2020-01-06-12-47-23 
 
[Container] 2020/01/07 15:34:53 Phase complete: BUILD State: SUCCEEDED 
[Container] 2020/01/07 15:34:53 Phase context status code:  Message:  
[Container] 2020/01/07 15:34:53 Entering phase POST_BUILD 
[Container] 2020/01/07 15:34:53 Phase complete: POST_BUILD State: SUCCEEDED 
[Container] 2020/01/07 15:34:53 Phase context status code:  Message:  
 

# Test Endpoint

## dev account

* Please login as mlops-dev-developers user

* mkdir test-end-point
* cd test-endpoint
* `curl https://raw.githubusercontent.com/awslabs/amazon-sagemaker-examples/master/advanced_functionality/scikit_bring_your_own/data/iris.csv --output iris.csv`

In [ ]:
curl https://raw.githubusercontent.com/awslabs/amazon-sagemaker-examples/master/advanced_functionality/scikit_bring_your_own/data/iris.csv --output iris.csv

## test-endpoint.py

In [ ]:
import boto3, io, itertools, sys
import pandas as pd

endpointName = sys.argv[1]
# Make a dataframe
shape = pd.read_csv('./iris.csv', header=None)

# Take a random sample
a = [50*i for i in range(3)]
b = [40+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]
test_data=shape.iloc[indices[:-1]]
test_X=test_data.iloc[:,1:]
test_y=test_data.iloc[:,0]

# Convert the dataframe to csv data
test_file = io.StringIO()
test_X.to_csv(test_file, header=None, index=None)

# Talk to SageMaker
client = boto3.Session(profile_name='mlops-prod-sagemaker').client('sagemaker-runtime')
response = client.invoke_endpoint(
    EndpointName=endpointName,
    Body=test_file.getvalue(),
    ContentType='text/csv',
    Accept='Accept'
)

print(response['Body'].read().decode('ascii'))

## config mlops-prod-sagemaker

* vim ~/.aws/config
* vim ~/.aws/credentials
* sudo easy_install-3.6 pip
* sudo /usr/local/bin/pip3 install boto3 pandas
* python3 test-endpoint-sample.py scikit-bring-your-own-2020-01-06-12-47-23

In [ ]:
mlops-dev-developers:~/environment/test-endpoint $ python3 test-endpoint-sample.py scikit-bring-your-own-2020-01-08-09-08-49
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
versicolor
versicolor
versicolor
versicolor

# Congradulation

Finish all parts

![](./img/arch.png)

# Clean Up

* Delete prod endpiont